In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator(verbose=False).build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count(), number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(1000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9986257788206535, 0.9984786152107649)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.999581621413139, 0.9995522839992403)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe(max_rows=10)

In [9]:
y

,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,NaN,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,NaN,Error,Boolean
1,ItalianVAT,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,Error,CountryCode,Name,Surname,NaN,EMail,PhoneNumber,NaN,BiologicalSex,Boolean
2,ItalianFiscalCode,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,NaN,Municipality,Year,...,Country,CountryCode,NaN,Surname,String,EMail,NaN,Date,BiologicalSex,NaN
3,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,NaN,NaN,NaN,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
4,NaN,NaN,Document,Plate,Address,NaN,NaN,Region,NaN,Year,...,Country,CountryCode,Name,Surname,String,NaN,PhoneNumber,Date,NaN,Boolean


In [10]:
model.predict_dataframe(X)

,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,NaN,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,NaN,Error,Boolean
1,ItalianVAT,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,Error,CountryCode,Name,Surname,NaN,EMail,PhoneNumber,NaN,BiologicalSex,Boolean
2,ItalianFiscalCode,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,NaN,Municipality,Year,...,Country,CountryCode,NaN,Surname,String,EMail,NaN,Date,BiologicalSex,NaN
3,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,NaN,NaN,NaN,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
4,NaN,NaN,Document,Plate,Address,NaN,NaN,Region,NaN,Year,...,Country,CountryCode,Name,Surname,String,NaN,PhoneNumber,Date,NaN,Boolean


In [11]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('ItalianVAT', 'Error'): 34,
         ('Error', 'Document'): 161,
         ('Error', 'CountryCode'): 25,
         ('CountryCode', 'NaN'): 76,
         ('Date', 'Error'): 5,
         ('Document', 'Error'): 104,
         ('Error', 'ItalianVAT'): 25,
         ('String', 'Error'): 78,
         ('Error', 'ItalianFiscalCode'): 14,
         ('NaN', 'CountryCode'): 81,
         ('ProvinceCode', 'Error'): 3,
         ('Error', 'Plate'): 5,
         ('Error', 'ProvinceCode'): 25,
         ('Error', 'Region'): 4,
         ('CountryCode', 'ProvinceCode'): 7,
         ('Error', 'String'): 16,
         ('Error', 'Date'): 7,
         ('Municipality', 'Error'): 11,
         ('String', 'Float'): 3,
         ('Name', 'Error'): 53,
         ('Surname', 'Error'): 9,
         ('Error', 'NaN'): 2,
         ('Error', 'Integer'): 2,
         ('Error', 'CadastreCode'): 3,
         ('Region', 'Error'): 3,
         ('Float', 'Error'): 3,
         ('Integer', 'Error'): 10,
         ('ItalianFiscalCode', 